## Team member and Contributions
> Shu Xu (shuxu3@illinois.edu): Part II/III

> Yan Han (yanhan4@illinois.edu): Part I

> Amrit Kumar(amritk2@illinois.edu): Part II

**We finish this notebook together.**

In [1]:
import numpy as np
import pandas as pd
import math

## Part I:  Gaussian Mixtures

### Objective
Implement the EM algorithm **from scratch** for a $p$-dimensional Gaussian mixture model 
with $G$ components: 
$$
\sum_{k=1}^G p_k \cdot \textsf{N}(x; \mu_k, \Sigma).
$$
### Requirements

Your implementation should consists of **four** functions. 

- **`Estep`** function: This function should return an $n$-by-$G$ matrix, where the $(i,j)$th entry represents the conditional probability $P(Z_i = k \mid x_i)$. Here $i$ ranges from 1 to $n$ and $k$ ranges from $1$ to $G$.

- **`Mstep`** function: This function should return the updated parameters for the Gaussian mixture model.

- **`loglik`** function: This function computes the log-likelihood of the data given the parameters.

- **`myEM`** function (main function): Inside this function, you can call the `Estep`, `Mstep`, and `loglik` functions. The function should take the following inputs and return the estimated parameters and log-likelihood:     

  - **Input**: 
    - data: The dataset.
    - $G$: The number of components.
    - Initial parameters.
    - `itmax`: The number of iterations.
  - **Output**: 
    - `prob`: A $G$-dimensional probability vector $(p_1, \dots, p_G)$
    - `mean`: A $p$-by-$G$ matrix with the $k$-th column being $\mu_k$, the $p$-dimensional mean for the $k$-th Gaussian component. 
    - `Sigma`: A $p$-by-$p$ covariance matrix $\Sigma$ shared by all $G$ components; 
    - `loglik`: A number equal to $\sum_{i=1}^n \log \Big [ \sum_{k=1}^G p_k \cdot \textsf{N}(x; \mu_k, \Sigma) \Big ].$

**Implementation Guidelines:**

  - Avoid explicit loops over the sample size $n$.
  - You are allowed to use loops over the number of components $G$, although you can avoid all loops. 
  - You are not allowed to use pre-existing functions or packages for evaluating normal densities.

In [2]:
def Estep(data, G, prob, mu, Sigma):
    g = np.zeros((len(data), G))
    U, D, _ = np.linalg.svd(Sigma)
    D_tilda = np.diag(1 / np.sqrt(D))
    x_tilda = D_tilda.dot(U.T.dot(data.T)).T # n by dim
    mu_tilda = D_tilda.dot(U.T.dot(mu)).T # G by dim
    for i in range(G):
        g[:, i] = np.sum(((x_tilda - mu_tilda[i])*(x_tilda - mu_tilda[i])), axis=1)
        g[:, i] = np.exp(-0.5 * g[:, i]) * prob[i]
    g /= np.sqrt((2*math.pi) ** ndim * np.linalg.det(Sigma))
    g /= np.sum(g, axis=1, keepdims=True)
    return g

In [3]:
def Mstep(data, g):
    prob = np.mean(g, axis=0)
    mu = np.zeros((ndim, G))
    Sigma = np.zeros((ndim, ndim))
    for i in range(G):
        mu[:, i] = np.sum(np.multiply(g[:, i:i+1], data), axis=0) / np.sum(g[:, i])
        temp_data = data - mu[:, i]
        temp_data1 = np.multiply(g[:, i:i+1], temp_data)
        Sigma += temp_data.T.dot(temp_data1)
    Sigma /= n
    return prob, mu, Sigma

In [4]:
def loglik(data, prob, mu, Sigma):
    g = np.zeros((len(data), G))
    U, D, _ = np.linalg.svd(Sigma)
    D_tilda = np.diag(1 / np.sqrt(D))
    x_tilda = D_tilda.dot(U.T.dot(data.T)).T # n by dim
    mu_tilda = D_tilda.dot(U.T.dot(mu)).T
    for i in range(G):
        g[:, i] = np.sum(((x_tilda - mu_tilda[i])*(x_tilda - mu_tilda[i])), axis=1)
        g[:, i] = np.exp(-0.5 * g[:, i]) * prob[i]
    g /= np.sqrt((2*math.pi) ** ndim * np.linalg.det(Sigma))
    llh = np.sum(np.log(np.sum(g, axis=1)), axis=0)
    return llh

In [5]:
def myEM(data, G, prob, mu, Sigma, itmax=20):
    for i in range(itmax):
        g = Estep(data, G, prob, mu, Sigma)
        prob, mu, Sigma = Mstep(data, g)
    return prob, mu, Sigma, loglik(data, prob, mu, Sigma)

### Testing

Test your code with the provided dataset,  [[faithful.dat](https://liangfgithub.github.io/Data/faithful.dat)], with both $G=2$ and $G=3$. 


In [6]:
data = pd.read_table("faithful.dat", sep="\s+", index_col=0)

**For the case when $G=2$**, set your initial values as follows:

- $p_1 = 10/n$, $p_2 = 1 - p_1$.
- $\mu_1$ =  the mean of the first 10 samples; $\mu_2$ = the mean of the remaining samples.
- Calculate $\Sigma$ as  
$$
\frac{1}{n} \Big [ \sum_{i=1}^{10} (x_i- \mu_1)(x_i- \mu_1)^t + \sum_{i=11}^n (x_i- \mu_2)(x_i- \mu_2)^t \Big].
$$
Here $x_i - \mu_i$ is a 2-by-1 vector, so the resulting $\Sigma$ matrix is a 2-by-2 matrix. 

Run your EM implementation with **20** iterations. 

In [7]:
n = len(data)
G = 2
ndim = 2
prob = np.array([10/n, 1-10/n])
mu = np.array([np.mean(data[:10], axis=0), np.mean(data[10:], axis=0)])
mu = mu.T
Sigma = np.zeros((2,2))
temp_data1 = np.array(data[:10]-mu[:, 0])
temp_data2 = np.array(data[10:]-mu[:, 1])
Sigma[0, 0] = np.sum(np.multiply(temp_data1[:,0], temp_data1[:,0])) + np.sum(np.multiply(temp_data2[:,0], temp_data2[:,0]))
Sigma[0, 1] = Sigma[1, 0] = np.sum(np.multiply(temp_data1[:,0],temp_data1[:,1]))+np.sum(np.multiply(temp_data2[:,0],temp_data2[:,1]))
Sigma[1, 1] = np.sum(np.multiply(temp_data1[:,1], temp_data1[:,1])) + np.sum(np.multiply(temp_data2[:,1], temp_data2[:,1]))
Sigma /= n

In [8]:
new_prob, new_mu, new_Sigma, llh = myEM(data, G, prob, mu, Sigma, itmax=20)

In [9]:
print("prob")
print(new_prob)
print("mean")
print(new_mu)
print("Sigma")
print(new_Sigma)
print("loglik")
print(llh)

prob
[0.04297883 0.95702117]
mean
[[ 3.49564188  3.48743016]
 [76.79789154 70.63205853]]
Sigma
           eruptions     waiting
eruptions   1.297936   13.924336
waiting    13.924336  182.580092
loglik
-1289.5693549424138


**For the case when $G=3$**, set your initial values as follows:


- $p_1 = 10/n$, $p_2 = 20/n$, $p_3= 1 - p_1 - p_2$
- $\mu_1 = \frac{1}{10} \sum_{i=1}^{10} x_i$,  the mean of the first 10 samples; $\mu_2 = \frac{1}{20} \sum_{i=11}^{30} x_i$, the mean of next 20 samples; and $\mu_3$ = the mean of the remaining samples. 
- Calculate $\Sigma$ as 
$$
\frac{1}{n} \Big [ \sum_{i=1}^{10} (x_i- \mu_1)(x_i- \mu_1)^t + \sum_{i=11}^{30} (x_i- \mu_2)(x_i- \mu_2)^t + \sum_{i=31}^n (x_i- \mu_3)(x_i- \mu_3)^t \Big].$$


Run your EM implementation with **20** iterations. 

In [10]:
n = len(data)
G = 3
ndim = 2
prob = np.array([10/n, 20/n, 1-10/n-20/n])
mu = np.array([np.mean(data[:10], axis=0), np.mean(data[10:30], axis=0),
np.mean(data[30:], axis=0)])
mu = mu.T
Sigma = np.zeros((ndim,ndim))
temp_data1 = np.array(data[:10]-mu[:, 0])
temp_data2 = np.array(data[10:30]-mu[:, 1])
temp_data3 = np.array(data[30:]-mu[:, 2])
Sigma[0, 0] = np.sum(np.multiply(temp_data1[:,0], temp_data1[:,0])) + np.sum(np.multiply(temp_data2[:,0], temp_data2[:,0])) + np.sum(np.multiply(temp_data3[:,0], temp_data3[:,0]))
Sigma[0, 1] = Sigma[1, 0] = np.sum(np.multiply(temp_data1[:,0],temp_data1[:,1]))+np.sum(np.multiply(temp_data2[:,0],temp_data2[:,1]))+np.sum(np.multiply(temp_data3[:,0],temp_data3[:,1]))
Sigma[1, 1] = np.sum(np.multiply(temp_data1[:,1], temp_data1[:,1])) + np.sum(np.multiply(temp_data2[:,1], temp_data2[:,1])) + np.sum(np.multiply(temp_data3[:,1], temp_data3[:,1]))
Sigma /= n

In [11]:
new_prob, new_mu, new_Sigma, llh = myEM(data, G, prob, mu, Sigma, itmax=20)

In [12]:
print("prob")
print(new_prob)
print("mean")
print(new_mu)
print("Sigma")
print(new_Sigma)
print("loglik")
print(llh)

prob
[0.04363422 0.07718656 0.87917922]
mean
[[ 3.51006918  2.81616674  3.54564083]
 [77.10563811 63.35752634 71.25084801]]
Sigma
           eruptions     waiting
eruptions   1.260158   13.511538
waiting    13.511538  177.964191
loglik
-1289.3509588627353


## Part II: HMM
**Objective**
Implement the `Baum-Welch` (i.e., EM) algorithm and the `Viterbi` algorithm from scratch for a Hidden Markov Model (HMM) that produces an outcome sequence of discrete random variables with three distinct values.
A quick review on parameters for Discrete HMM:

- `mx`: Count of distinct values X can take.
- `mz`: Count of distinct values Z can take.
- `w`: An mz-by-1 probability vector representing the initial distribution for Z1.
- `A`: The mz-by-mz transition probability matrix that models the progression from Zt to Zt+1.
- `B`: The mz-by-mx emission probability matrix, indicating how X is produced from Z.

Focus on updating the parameters `A` and `B` in your algorithm. The value for `mx` is given and you’ll specify `mz`.

For `w`, initiate it uniformly but refrain from updating it within your code. The reason for this is that `w` denotes the distribution of Z1 and we only have a single sample. It’s analogous to estimating the likelihood of a coin toss resulting in heads by only tossing it once. Given the scant information and the minimal influence on the estimation of other parameters, we can skip updating it.

**Baum-Welch Algorihtm**
The Baum-Welch Algorihtm is the EM algorithm for the HMM. Create a function named `BW.onestep` designed to carry out the E-step and M-step. This function should then be called iteratively within `myBW`.

`BW.onstep`:
- **Input**:
    – data: a T-by-1 sequence of observations
    – Current parameter values
- **Output**:
    – Updated parameters: A and B
Please refer to formulas provided on Pages 7, 10, 14-16 in [lec_W7.2_HMM](https://liangfgithub.github.io/Notes/lec_W7.2_HMM.pdf)

In [25]:
# Load the data sequence from the file
# X: i.e., states,  start from 0 in python
X = np.genfromtxt("coding4_part2_data.txt", dtype=int) - 1
w = np.array([0.5, 0.5])
A = np.array([[0.5, 0.5], [0.5, 0.5]])
B = np.array([[1.0/9.0, 3.0/9.0, 5.0/9.0], [1.0/6.0, 2.0/6.0, 3.0/6.0]])

In [26]:
# Compute alphas
def compute_forward_probability(A, B, X, w):
    T = len(X)  # Get the length of the sequence X
    mz = A.shape[0]
    # Initialize the forward probability matrix
    alpha = np.zeros((T, mz))
    
    # Calculate the forward probabilities recursively
    for t in range(T):
        if t == 0:
            # At t = 1, use the initial distribution w and emission probability B
            alpha[t, :] = w * B[:, X[t]]
            #print(f" t: {t} , alpha[t] : {alpha[t] }")
        else:
            # For t > 1, calculate alpha using the previous time step's alpha, transition probability A, and emission probability B
            for i in range(mz):
                # Using .dot to replace summation over j
                alpha[t, i] = (np.dot(alpha[t - 1, :], A[:, i])) * B[i, X[t]]
                #print(f" t: {t} , i:{i}, alpha[{t},{i}] : {alpha[t,i] }")
    return alpha

In [27]:
# Compute betas
def compute_backward_probability(A, B, X):
    
    T = len(X)
    mz = A.shape[0]
    beta = np.zeros((T, mz))

    # Iterate backwards
    for t in range(T-1,-1,-1):
        if t == T-1:
            # Last beta equals ones
            beta[t, :] = np.ones(mz)
        else:    
            for i in range(mz):
                # Using .dot to replace summation over j
                beta[t, i] = np.dot(beta[t+1, :]*B[:,X[t+1]], A[i,:])
 
    return beta

In [28]:
# Baum-Welch Algorithm One Step
def BW_one_step(A, B, X, w):
    
    T = len(X)
    mz = A.shape[0]
    mx = B.shape[1]
    myGamma = np.zeros((T-1, mz, mz))
    gamma_t = np.zeros((T, mz))    
        
    alpha = compute_forward_probability(A, B, X, w)
    beta =  compute_backward_probability(A, B, X)   

    # To update gamma_t, first calculate gammas on Slide#16    
    for i in range(mz):
        for j in range(mz):
            for t in range(T-1):
                myGamma[t,i,j] = alpha[t,i]*A[i,j]*B[j,X[t+1]]*beta[t+1,j]
    
    # update gamma_t based on gamma
    for i in range(mz):
        for t in range(T-1):
            gamma_t[t,i] = np.sum(myGamma[t,i,:])

    gamma_t[T-1, :] = gamma_t[T-2, :]
    
    # Update A_{mz x mz}
    for i in range(mz):
        for j in range(mz):
            A[i,j] = np.sum(myGamma[:,i,j])
        
        A[i,:] = A[i,:]/np.sum(A[i, :])
        
    # Update B_{mz x mz}
    for i in range(mz):
        for l in range(mx):
            B[i,l] = 0
            for t in range(T):
                B[i,l] += (X[t]==l)*(gamma_t[t,i])
        B[i,:] = B[i,:]/np.sum(B[i, :])
    
    return A, B

In [29]:
A1, B1 = BW_one_step(A, B, X, w)
print(f'A matrix after BW one step is')
print(A1)

print(f'B matrix after BW one step is')
print(B1)

A matrix after BW one step is
[[0.485465   0.514535  ]
 [0.48503179 0.51496821]]
B matrix after BW one step is
[[0.2348986  0.19574883 0.56935256]
 [0.33224256 0.1845792  0.48317824]]


In [30]:
# Baum-Welch Algorithm multiple steps
def myBW(Ai, Bi, X, w, n_iter):
    A = Ai.copy()
    B = Bi.copy()
    for iter in range(n_iter):
        A, B = BW_one_step(A, B, X, w)
    return A, B

In [31]:
A100, B100 = myBW(A, B, X, w, 100)
print(f'A matrix after BW 100th step is')
print(A100)

print(f'B matrix after BW 100th step is')
print(B100)

A matrix after BW 100th step is
[[0.49756056 0.50243944]
 [0.44840436 0.55159564]]
B matrix after BW 100th step is
[[0.22122219 0.20345148 0.57532633]
 [0.34199559 0.17797897 0.48002545]]


## Part III Viterbi Algorithm
This algorithm outputs the most likely latent sequence considering the data and the MLE of the parameters.
`myViterbi`:
- **Input**:

    – data: a T-by-1 sequence of observations
    
    – parameters: `mx`, `mz`, `w`, `A` and `B`:
    - `mx`: Count of distinct values X can take.
    - `mz`: Count of distinct values Z can take.
    -  `w`: An mz-by-1 probability vector representing the initial distribution for Z1.
    - `A`: The mz-by-mz transition probability matrix that models the progression from Zt to Zt+1.
    - `B`: The mz-by-mx emission probability matrix, indicating how X is produced from Z.

- **Output**:

    – Z: A T-by-1 sequence where each entry is a number ranging from 1 to mz.


#### Note on Calculations in Viterbi:

Many computations in HMM are based on the product of a sequence of probabilities, resulting in extremely small values. At times, these values are so small that software like R or Python might interpret them as zeros. This poses a challenge, especially for the Viterbi algorithm, where differentiating between magnitudes is crucial. If truncated to zero, making such distinctions becomes impossible. Therefore, it’s advisable to evaluate these probabilities on a logarithmic scale in the Viterbi algorithm.


#### Testing
Test your code with the provided data sequence: [Coding4_part2_data.txt]. Set `mz = 2` and start with the following initial values
$$
    w = \begin{pmatrix}     
        0.5 \\
        0.5 
        \end{pmatrix}


    A = \begin{pmatrix}     
        0.5 & 0.5\\
        0.5 & 0.5
        \end{pmatrix} 


    B = \begin{pmatrix}     
        1/9 & 3/9 & 5/9\\
        1/6 & 2/6 & 3/6
        \end{pmatrix}        
$$,
Run your implementation with **100** iterations

In [20]:
def viterbi(data, A, B, w,  mx=3, mz=2):
    T = len(data)
    delta = np.zeros((T, mz))

    log_A = np.log(A)
    log_B = np.log(B)
    log_w = np.log(w)


    # Compute delta
    delta[0,:] = log_w  +  log_B[:, data[0]]

    for t in range(1, T):
        for i in range(mz):
            # -1 because python index starts from 0
            delta[t,i] = np.max(delta[t-1,:] + log_A[:,i]) + log_B[i, data[t]-1]

    # compute the most prob sequence Z
    Z = np.zeros(T).astype(int)

    # start from the end

    Z[T-1] = np.argmax(delta[T-1, :])

    for t in range(T-2, -1, -1):
        Z[t] = np.argmax(delta[t, :] + log_A[:, Z[t+1]])    

    # +1: because python index start from 0
    return Z + 1

In [21]:
data = np.squeeze(pd.read_csv('coding4_part2_data.txt', header=None).values)

A_bw = np.array([[0.49793938, 0.50206062],
            [0.44883431, 0.55116569]])

B_bw = np.array([[0.22159897, 0.20266127, 0.57573976],
               [0.34175148, 0.17866665, 0.47958186]])

w = np.array([0.5, 0.5])
# Estimated states
Z_est = viterbi(data, A_bw, B_bw, w)
print(Z_est)

[1 1 1 1 1 1 1 2 1 1 1 1 1 2 2 1 1 1 1 1 1 1 2 2 2 2 2 1 1 1 1 1 1 1 2 1 1
 1 1 1 1 1 1 2 2 1 1 1 1 1 1 2 2 2 1 1 1 1 2 2 2 2 1 1 1 1 1 1 1 1 2 2 2 2
 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 1 1 1 1 2 1 1 1 1 1 1 1 1 1 1 1 1 1 2 1
 1 1 1 2 1 1 1 1 1 1 2 2 2 2 2 2 2 2 2 2 2 2 1 1 1 1 1 1 1 2 2 2 2 2 2 2 2
 2 2 2 1 1 1 2 2 2 2 2 2 1 1 1 1 1 2 2 2 2 2 2 2 2 2 1 1 1 2 2 2 1 1 1 1 1
 1 1 1 2 2 2 2 2 1 1 1 1 1 1 1]


In [22]:
# Actual states
Z_act_list = np.squeeze(pd.read_csv('Coding4_part2_Z.txt',header=None).values)
Z_act_str_list = []
for z_i in Z_act_list:
    z_i_str_list = z_i.strip().split(" ")
    Z_act_str_list.extend(z_i_str_list)

Z_act = np.array([int(z) for z in Z_act_str_list])
print(Z_act)

[1 1 1 1 1 1 1 2 1 1 1 1 1 2 2 1 1 1 1 1 1 1 2 2 2 2 2 1 1 1 1 1 1 1 2 1 1
 1 1 1 1 1 1 2 2 1 1 1 1 1 1 2 2 2 1 1 1 1 2 2 2 2 1 1 1 1 1 1 1 1 2 2 2 2
 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 1 1 1 1 2 1 1 1 1 1 1 1 1 1 1 1 1 1 2 1
 1 1 1 2 1 1 1 1 1 1 2 2 2 2 2 2 2 2 2 2 2 2 1 1 1 1 1 1 1 2 2 2 2 2 2 2 2
 2 2 2 1 1 1 2 2 2 2 2 2 1 1 1 1 1 2 2 2 2 2 2 2 2 2 1 1 1 2 2 2 1 1 1 1 1
 1 1 1 2 2 2 2 2 1 1 1 1 1 1 1]


In [23]:
# The estimated states match the actual ones
print((Z_act == Z_est).all())

True
